<a href="https://colab.research.google.com/github/fay23-dam/cloud_computing_tugas/blob/main/Salinan_dari_RAG_Question_Answering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Persiapan dan Load data

In [ ]:
!pip install transformers datasets faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:
from transformers import DistilBertTokenizer, DistilBertModel
from datasets import load_dataset
import numpy as np
import faiss
import torch

# Load tokenizer dan model DistilBERT
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")
model = DistilBertModel.from_pretrained("distilbert-base-uncased")

# Load dataset Sales Textbook
dataset = load_dataset("goendalf666/sales-textbook_for_convincing_and_selling")
texts = [item["text"] for item in dataset["train"]]

texts

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

README.md:   0%|          | 0.00/1.81k [00:00<?, ?B/s]

sales_textbook.txt:   0%|          | 0.00/460k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/1461 [00:00<?, ? examples/s]

['Chapter 1: Building Rapport and Capturing Attention',
 'Subpoint: Understanding the Importance of Building Rapport',
 'Building rapport is a fundamental skill in sales that cannot be underestimated. It lays the foundation for establishing a connection with your potential customers, gaining their trust, and ultimately convincing them to make a purchase. Rapport can be defined as a harmonious relationship based on mutual understanding and empathy. When you build rapport with someone, you create a sense of familiarity, comfort, and shared interests, making it easier to communicate and influence their decision-making process.',
 'One of the main reasons why building rapport is crucial in sales is that people are more likely to buy from someone they like and trust. By establishing a positive and genuine connection with your customers, you increase their confidence in you and your product or service. People want to do business with individuals they feel comfortable with, those who understa

In [ ]:
# Buka file untuk ditulis
with open('texts.txt', 'w', encoding='utf-8') as f:
    for text in texts:
        f.write(text + '\n')

## Latih data teks untuk retreival

In [ ]:
# Fungsi untuk menghasilkan embedding menggunakan DistilBERT
def get_embeddings(texts, tokenizer, model, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i+batch_size]
        encoded_input = tokenizer(batch, padding=True, truncation=True, return_tensors="pt", max_length=512)
        with torch.no_grad():
            output = model(**encoded_input)
            # Gunakan mean pooling dari output terakhir
            embedding = output.last_hidden_state.mean(dim=1).cpu().numpy()
        embeddings.append(embedding)
    return np.vstack(embeddings)

# Convert teks menjadi embedding
data_embeddings = get_embeddings(texts, tokenizer, model)

# Dimensi embedding (harus sama dengan output model)
d = data_embeddings.shape[1]

# Buat FAISS index
index = faiss.IndexFlatL2(d)
index.add(data_embeddings)
print("Jumlah data dalam index:", index.ntotal)

Jumlah data dalam index: 1461


In [ ]:
data_embeddings

array([[ 0.12001564, -0.15674254,  0.2033178 , ..., -0.1337743 ,
        -0.03100199, -0.10861351],
       [ 0.0183659 , -0.05255767,  0.06753943, ..., -0.1593043 ,
        -0.10101003,  0.086523  ],
       [-0.10518011,  0.16062666,  0.31301615, ..., -0.2554612 ,
        -0.2332608 ,  0.1649919 ],
       ...,
       [ 0.16390125,  0.01669998,  0.27401075, ..., -0.10487692,
        -0.06015566, -0.05285979],
       [ 0.08626136,  0.3051466 ,  0.35237244, ..., -0.17474091,
        -0.08657936, -0.28974238],
       [-0.07986715,  0.3587254 ,  0.38200524, ..., -0.24201767,
        -0.09616397, -0.06391858]], dtype=float32)

## Retreival Query

In [ ]:
# Fungsi untuk konversi query ke embedding
def query_to_embeddings(query, tokenizer, model):
    encoded_input = tokenizer([query], padding=True, truncation=True, return_tensors="pt", max_length=512)
    with torch.no_grad():
        output = model(**encoded_input)
        embedding = output.last_hidden_state.mean(dim=1).cpu().numpy()
    return embedding

# Fungsi untuk melakukan pencarian
def retrieve(query, index, texts, k):
    query_embedding = query_to_embeddings(query, tokenizer, model)
    distances, indices = index.search(query_embedding, k)
    results = [texts[i] for i in indices[0]]  # Ambil teks dari indeks
    return results

# Contoh pencarian
query = "what skills are essential for sales professionals to succeed?"
top_k_results = retrieve(query, index, texts, k=5)
print("Top 5 Results:")
for i, res in enumerate(top_k_results):
    print(f"{i+1}. {res}")

Top 5 Results:
1. In the world of sales, building credibility and trust with your potential customers is crucial. Without these two essential elements, it becomes challenging to convince someone to buy a product or service. In this section, we will explore effective techniques that will help you establish credibility and trust, enabling you to create strong connections with your customers.
2. In the business world, negotiation and conflict resolution skills are essential for sales professionals to succeed. This subpoint focuses on equipping individuals with the knowledge and techniques required to handle difficult situations, resolve conflicts, and negotiate effectively. By mastering these skills, salespeople can navigate challenging scenarios and secure successful outcomes for both themselves and their customers.
3. Sales professionals need to recognize the potential benefits of overcoming challenges and obstacles. By highlighting the positive outcomes that can result from addressing 

In [ ]:
# Contoh pencarian 2
query = "How to create a positive first impression?"
top_k_results = retrieve(query, index, texts, k=5)
print("Top 5 Results:")
for i, res in enumerate(top_k_results):
    print(f"{i+1}. {res}")

Top 5 Results:
1. Creating a positive first impression is not just about superficial charm or manipulation; it is about genuinely caring for the customer and their needs. It is about establishing a strong foundation of trust and credibility, setting the stage for a successful sales journey. By mastering the art of creating a positive first impression, you lay the groundwork for building lasting relationships and achieving sales success. Chapter 1: Building Rapport and Capturing Attention
2. Another effective way to create a positive first impression is by engaging in small talk and finding common ground with the customer. This can be achieved by researching their background, interests, or industry beforehand. By demonstrating knowledge and showing a genuine interest in their world, you establish a connection and make them feel valued as an individual. Finding common ground also helps to build rapport and creates a foundation for a more meaningful conversation.
3. 3. Empathy and Underst

## Generator

In [ ]:
from transformers import pipeline

# Pipeline untuk GPT-2 untuk Question Answering
qa_pipeline = pipeline('question-answering', model="openai-community/gpt2-large", tokenizer="openai-community/gpt2-large")

config.json:   0%|          | 0.00/666 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.25G [00:00<?, ?B/s]

Some weights of GPT2ForQuestionAnswering were not initialized from the model checkpoint at openai-community/gpt2-large and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


In [ ]:
# Fungsi untuk menghasilkan jawaban menggunakan pipeline
def generate_answer(query, context):
    result = qa_pipeline({
        'question': query,
        'context': context
    })
    return result['answer']

# Contoh penggunaan
query = "How to create a positive first impression?"
top_k_results = retrieve(query, index, texts, 1)
relevant_doc = top_k_results[0]
answer = generate_answer(query, relevant_doc)

print("Question :", query)
print("Context :",relevant_doc)
print("Generated Answer:", answer)

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/question_answering.py:391: FutureWarning: Passing a list of SQuAD examples to the pipeline is deprecated and will be removed in v5. Inputs should be passed using the `question` and `context` keyword arguments instead.
  warnings.warn(


Question : How to create a positive first impression?
Context : Creating a positive first impression is not just about superficial charm or manipulation; it is about genuinely caring for the customer and their needs. It is about establishing a strong foundation of trust and credibility, setting the stage for a successful sales journey. By mastering the art of creating a positive first impression, you lay the groundwork for building lasting relationships and achieving sales success. Chapter 1: Building Rapport and Capturing Attention
Generated Answer:  success. Chapter 1: Building Rapport and Capturing


In [ ]:
# Contoh penggunaan 2
query = "What adventage of storytelling and anecdotes for sales?"
top_k_results = retrieve(query, index, texts, 1)
relevant_doc = top_k_results[0]
answer = generate_answer(query, relevant_doc)

print("Question :", query)
print("Context :",relevant_doc)
print("Generated Answer:", answer)

Question : What adventage of storytelling and anecdotes for sales?
Context : Humans are naturally drawn to stories, so incorporating storytelling into your sales pitch can be an incredibly powerful attention-grabbing technique. By sharing a relatable and engaging anecdote or case study, you can captivate your audience and create an emotional connection. Make sure the story highlights the benefits or outcomes your product or service can offer, leaving your prospects eager to hear more.
Generated Answer:  anecdote or case study, you can captivate


In [ ]:
# Contoh penggunaan 3
query = "Why is it important to use appropriate language and tone in sales communication?"
top_k_results = retrieve(query, index, texts, 1)
relevant_doc = top_k_results[0]
answer = generate_answer(query, relevant_doc)

print("Question :", query)
print("Context :",relevant_doc)
print("Generated Answer:", answer)

Question : Why is it important to use appropriate language and tone in sales communication?
Context : In written communication or over the phone, handling objections related to trust or credibility requires careful consideration of the language used. Sales professionals should employ clear and concise language, avoiding jargon or technical terms that may confuse or alienate the customer. By using a tone that is reassuring and informative, they can build trust and credibility even without the advantage of non-verbal cues.
Generated Answer:  tone that


In [ ]:
# Contoh penggunaan 4
query = "What is the important thing for sales?"
top_k_results = retrieve(query, index, texts, 1)
relevant_doc = top_k_results[0]
answer = generate_answer(query, relevant_doc)

print("Question :", query)
print("Context :",relevant_doc)
print("Generated Answer:", answer)

Question : What is the important thing for sales?
Context : In the world of sales, closing the sale is the ultimate goal. It is the moment where all your efforts in building rapport, understanding customer needs, and presenting solutions come together. However, closing techniques are not a one-size-fits-all approach. Different situations and customers require different approaches. In this section, we will explore the importance of choosing the most appropriate closing technique to maximize your chances of success.
Generated Answer:  closing the


In [ ]:
# Contoh penggunaan 5
query = "How to convince customers to buy some product or services?"
top_k_results = retrieve(query, index, texts, 1)
relevant_doc = top_k_results[0]
answer = generate_answer(query, relevant_doc)

print("Question :", query)
print("Context :",relevant_doc)
print("Generated Answer:", answer)

Question : How to convince customers to buy some product or services?
Context : 4. Offering guarantees and warranties: To further enhance credibility, consider offering guarantees or warranties for your products or services. This sends a message to customers that you believe in the quality and reliability of what you are selling. By standing behind your offering, you alleviate any doubts potential customers may have, making them more likely to trust and buy from you.
Generated Answer:  offering, you alleviate any
